# Tutorial 5: Generating data using Differential Privacy guarantees

`synthcity` includes several privacy-focused generators. One class of privacy models is for differential privacy.

The central idea behind `differential privacy` is to add random noise to the data in a way that preserves the overall statistical properties of the dataset, but makes it impossible to determine the presence or absence of any individual data point within the dataset. This is achieved by bounding the amount of information that any single query can reveal about an individual data point.

One key element in DP is __epsilon (ε)__, a measure of the privacy loss of a given algorithm or mechanism. Specifically, it is a way to quantify the maximum amount by which the probability of any outcome (or set of outcomes) can change due to the inclusion or exclusion of a single individual's data.

For example, suppose a given query to a differentially private algorithm has a privacy loss of ε=1. In that case, that means that the probability of any outcome (or set of outcomes) can change by, at most, a factor of e^1=2.718 (about 2.718 times) due to the inclusion or exclusion of a single individual's data. The lower the value of `ε`, the stronger the privacy guarantees provided by the algorithm or mechanism.

One common way to achieve differential privacy is through the use of randomization. A differentially private algorithm will add noise to the output of a query in order to make it difficult to determine whether any particular individual's data was used in the computation. The amount of noise added is typically determined by the value of ε, with a lower value of ε resulting in more noise being added and stronger privacy guarantees.

It is important to note that ε is not the only parameter that is used to measure privacy loss in differential privacy. Another commonly used parameter is __delta (δ)__, which represents the probability that the privacy loss of an algorithm or mechanism exceeds a certain threshold (often ε). Together, ε and δ can be used to specify the privacy guarantees of a differentially private algorithm or mechanism more precisely.


`synthcity` includes the following models with DP-focus:
 - `AdsGAN` - A GAN with an identifiability penalty
 - `PATEGAN` - A GAN which uses the Private Aggregation of Teacher Ensembles (PATE) framework to tightly bound the influence of any individual sample on the model.
 - `PrivBayes` - uses a Bayesian network to iteratively learn a set of low-dimensional conditional probabilities from noisy marginals.
 - `DPGAN` - A GAN which uses the DP-SGD optimizer for training the discriminator.
 
 PATEGAN, PrivBayes, DPGAN have the `epsilon` parameter, which can be customized.
 AdsGAN takes a different approach, and the privacy level there can be controlled by the `lambda_identifiability_penalty` parameter.

In [ ]:
!pip install synthcity
!pip uninstall -y torchaudio torchdata

In [ ]:
# stdlib
import sys
import warnings

warnings.filterwarnings("ignore")

# third party
from sklearn.datasets import load_diabetes

# synthcity absolute
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

log.add(sink=sys.stderr, level="INFO")

X, y = load_diabetes(return_X_y=True, as_frame=True)
X["target"] = y

X

In [ ]:
# Note: preprocessing data with OneHotEncoder or StandardScaler is not needed or recommended. Synthcity handles feature encoding and standardization internally.
loader = GenericDataLoader(
    X,
    target_column="target",
    sensitive_columns=["sex"],
)

## List privacy plugins

In [ ]:
Plugins(categories=["privacy"]).list()

## Load and train a generative model

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

syn_model = Plugins().get("dpgan")

syn_model.fit(loader)

## Generate new data using the model

In [ ]:
syn_model.generate(count=10).dataframe()

## Benchmarking metrics

| **Metric**                                         | **Description**                                                                                                            |
|----------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------|
| sanity.data\_mismatch.score                        | Data types mismatch between the real//synthetic features                                                                   |
| sanity.common\_rows\_proportion.score              | Real data copy-paste in the synthetic data                                                                                 |
| sanity.nearest\_syn\_neighbor\_distance.mean       | Computes the \textless{}reduction\textgreater{}(distance) from the real data to the closest neighbor in the synthetic data |
| sanity.close\_values\_probability.score            | the probability of close values between the real and synthetic data.                                                       |
| sanity.distant\_values\_probability.score          | the probability of distant values between the real and synthetic data.                                                     |
| stats.jensenshannon\_dist.marginal                 | the average Jensen-Shannon distance                                                                                        |
| stats.chi\_squared\_test.marginal                  | the one-way chi-square test.                                                                                               |
| stats.feature\_corr.joint                          | the correlation/strength-of-association of features in data-set with both categorical and continuous features              |
| stats.inv\_kl\_divergence.marginal                 | the average inverse of the Kullback–Leibler Divergence metric.                                                             |
| stats.ks\_test.marginal                            | the Kolmogorov-Smirnov test for goodness of fit.                                                                           |
| stats.max\_mean\_discrepancy.joint                 | Empirical maximum mean discrepancy. The lower the result the more evidence that distributions are the same.                |
| stats.prdc.precision                               | precision between the two manifolds                                                                                        |
| stats.prdc.recall                                  | recall between the two manifolds                                                                                           |
| stats.prdc.density                                 | density between the two manifolds                                                                                          |
| stats.prdc.coverage                                | coverage between the two manifolds                                                                                         |
| stats.alpha\_precision.delta\_precision\_alpha\_OC | Delta precision                                                                                                            |
| stats.alpha\_precision.delta\_coverage\_beta\_OC   | Delta coverage                                                                                                             |
| stats.alpha\_precision.authenticity\_OC            | Authetnticity                                                                                                              |
| performance.linear\_model.gt.aucroc              | Train on real, test on the test real data using LogisticRegression: AUCROC                                                             |
| performance.linear\_model.syn\_id.aucroc         | Train on synthetic, test on the train real data using LogisticRegression: AUCROC                                                       |
| performance.linear\_model.syn\_ood.aucroc        | Train on synthetic, test on the test real data using LogisticRegression: AUCROC                                                        |
| performance.mlp.gt.aucroc                        | Train on real, test on the test real data using NN: AUCROC                                                                |
| performance.mlp.syn\_id.aucroc                    | Train on synthetic, test on the train real data using NN: AUCROC                                                          |
| performance.mlp.syn\_ood.aucroc                   | Train on synthetic, test on the test real data using NN: AUCROC                                                           |
| performance.xgb.gt.aucroc                         | Train on real, test on the test real data using XGB: AUCROC                                                               |
| performance.xgb.syn\_id.aucroc                    | Train on synthetic, test on the train real data using XGB: AUCROC                                                         |
| performance.xgb.syn\_ood.aucroc                   | Train on synthetic, test on the test real data using XGB: AUCROC                                                          |
| performance.feat\_rank\_distance.corr              | Correlation for the rank distances between the feature importance on real and synthetic data                               |
| performance.feat\_rank\_distance.pvalue            | p-vale for the rank distances between the feature importance on real and synthetic data                                    |
| detection.detection\_xgb.mean                      | The average AUCROC score for detecting synthetic data using an XGBoost.                                                    |
| detection.detection\_mlp.mean                      | The average AUCROC score for detecting synthetic data using a NN.                                                          |
| detection.detection\_gmm.mean                      | The average AUCROC score for detecting synthetic data using a GMM.                                                         |
| privacy.delta-presence.score                       | the maximum re-identification probability on the real dataset from the synthetic dataset.                                  |
| privacy.k-anonymization.gt                         | the k-anon for the real data                                                                                               |
| privacy.k-anonymization.syn                        | the k-anon for the synthetic data                                                                                          |
| privacy.k-map.score                                | the minimum value k that satisfies the k-map rule.                                                                         |
| privacy.distinct l-diversity.gt                    | the l-diversity for the real data                                                                                          |
| privacy.distinct l-diversity.syn                   | the l-diversity for the synthetic data                                                                                     |
| privacy.identifiability\_score.score               | the re-identification score on the real dataset from the synthetic dataset.                                                |

## Benchmark the quality of DPGAN for various epsilons

In [ ]:
# synthcity absolute
from synthcity.benchmark import Benchmarks

score = Benchmarks.evaluate(
    [(f"test_eps_{eps}", "dpgan", {"epsilon": eps}) for eps in [0.1, 1, 10]],
    loader,
    synthetic_size=1000,
    repeats=2,
    synthetic_reuse_if_exists=False
)

In [ ]:
Benchmarks.print(score)

## Benchmark the quality of DP-models for epsilon=0.1

In [ ]:
# synthcity absolute
from synthcity.benchmark import Benchmarks

score = Benchmarks.evaluate(
    [
        (f"test_{model}", model, {"epsilon": 0.1})
        for model in ["pategan", "dpgan"]
    ],
    loader,
    synthetic_size=1000,
    repeats=2,
    synthetic_reuse_if_exists=False,
)

In [ ]:
Benchmarks.print(score)

## Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star [Synthcity](https://github.com/vanderschaarlab/synthcity) on GitHub

- The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.


### Checkout other projects from vanderschaarlab
- [HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
- [AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
